In [1]:
import numpy as np

#Активационная функция сигмоидального нейрона
def sigmoid(x):
    return 1/(1+np.exp(-x))
    
#Производная активационной функции сигмоидального нейрона
def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

def _max(x):
    return np.max(x,0)

def max_prime(x):
    return int(x>0)

#Вычисляет вектор частных производных целевой функции по каждому из предсказаний(используется при расчете градиента)
def J_quadratic_derivative(y, y_hat):

    assert y_hat.shape == y.shape and y_hat.shape[1] == 1, 'Incorrect shapes'
    
    return (y_hat - y) / len(y)
    
    
#Рассчитывает градиент(аналитическая производная целевой функции)
def compute_grad_analytically(neuron, X, y, J_prime=J_quadratic_derivative):
    
    # Вычисляем активации
    # z - вектор результатов сумматорной функции нейрона на разных примерах
    
    z = neuron.summatory(X)
    y_hat = neuron.activation(z)

    # Вычисляем нужные нам частные производные
    dy_dyhat = J_prime(y, y_hat)
    dyhat_dz = neuron.activation_function_derivative(z)
    
    # осознайте эту строчку:
    dz_dw = X

    

    # а главное, эту:
    grad = ((dy_dyhat * dyhat_dz).T).dot(dz_dw)
    
    # можно было написать в два этапа. Осознайте, почему получается одно и то же
    #2)grad_matrix = dy_dyhat * dyhat_dz * dz_dw
    #2)grad = np.sum(grad_matrix, axis=0)
    
    # Сделаем из горизонтального вектора вертикальный
    
    
    grad = grad.T
    #2)grad.shape = (5,1)
    
    return grad

def compute_error_analytically(neuron, X, y, J_prime=J_quadratic_derivative):
     # Вычисляем активации
    # z - вектор результатов сумматорной функции нейрона на разных примерах
    
    z = neuron.summatory(X)
    y_hat = neuron.activation(z)

    # Вычисляем нужные нам частные производные
    dy_dyhat = J_prime(y, y_hat)
    dyhat_dz = neuron.activation_function_derivative(z)
    

    # а главное, эту:
    error = (dy_dyhat * dyhat_dz).T

    # Сделаем из горизонтального вектора вертикальный
    
    
    error = error.T
    
    return error

class Neuron:

    def __init__(self, weights, activation_function = sigmoid, activation_function_derivative = sigmoid_prime):
        
        #assert weights.shape[1] == 1, "Incorrect weight shape"
        
        self.w = weights
        self.activation_function = activation_function
        self.activation_function2 = _max
        self.activation_function_derivative = activation_function_derivative
        self.activation_function_derivative2 = max_prime

    #Сумматорный метод
    def summatory(self, input_matrix):
        return input_matrix.dot(self.w)
        
    #Активационный метод
    def activation(self, summatory_activation):
        #print("s: ", self.activation_function(summatory_activation))
        return self.activation_function(summatory_activation)
        
    #Векторизованный метод рассчета предсказанных значений(гораздо лучший способ)
    def vectorized_forward_pass(self, input_matrix):
        return self.activation(self.summatory(input_matrix))
    
    
    #Обучение весов с помощью одного батча(несколько случайных примеров из всех имеющихся примеров)
    def update_mini_batch(self, X, y, learning_rate, eps):
        
        before = J_quadratic(self, X, y)
        
        #self.w -= learning_rate * compute_grad_analytically(self,X,y)
        self.w -= learning_rate * compute_grad_numerically_2(self,X,y)

        
        return np.abs(before-J_quadratic(self, X, y)) < eps
    
    
    #Обучение весов с помощью батчей
    def SGD(self, X, y, batch_size, learning_rate =0.1, eps=1e-6, max_steps = 200):
        cur_step = 0
        #for _ in range(max_steps):
        while cur_step < max_steps:
            cur_step += 1
            batch = np.random.choice(len(X), batch_size, replace = False)
            if self.update_mini_batch(X[batch], y[batch], learning_rate, eps) == 1: 
                return 1
        return 0
                

def get_error_l3(a, y):
    
    return (a - y) * a * (1-a)
    
def get_error_l2(deltas, sums, weights, activation_func = sigmoid_prime):
    return (weights.reshape(len(weights),1).dot(deltas.reshape(1,deltas.shape[0])) * activation_func(sums.T)).mean(axis = 1)#более быстрое решение
    #return (weights.T.dot(deltas.reshape(1,deltas.shape[0])) * sigmoid_prime(sums.T)).mean(axis = 1)#более быстрое решение
    #return (deltas.dot(weights) * sigmoid_prime(sums)).mean(axis = 0).T


def FindWeight(l1,l2,l3,y):
    
    sumsL2 = l2.summatory(l1)
    aL2 = np.array((l2.activation_function2(sumsL2[0][0]), l2.activation_function(sumsL2[0][1]))).reshape(1,2)
    print("activation L2: \n", aL2, '\n')
    
    sumsL3 = l3.summatory(aL2).T
    aL3 = l3.activation_function(sumsL3).T
    print("activation L3: \n", aL3, '\n')
    
    errorL3 = get_error_l3(aL3, y)
    print("error L3: \n", errorL3, '\n')
    
    errorL2 = np.array((get_error_l2(errorL3, sumsL2[0][0], l3.w[0], activation_func = max_prime),get_error_l2(errorL3, sumsL2[0][1], l3.w[1]))).reshape(2)
    print("error L2: \n", errorL2, '\n')
    
    dJ_dw1_3 = l1[0][2]*errorL2[0]
    print("dJ_dw1_3: ", dJ_dw1_3)
    dJ_dw2_3 = l1[0][2]*errorL2[1]
    print("dJ_dw2_3: ", dJ_dw2_3)
    
    pass

w = np.array([[0.7,0.2,0.7], [0.8 ,0.3 ,0.6]])
w = w.T

w1 = np.array([[0.2],[0.4]])
x = np.array([[0.0,1.0,1.0]])
y = np.array([[1]])
layer2 = Neuron(w)
layer3 = Neuron(w1)

FindWeight(x, layer2, layer3, y)


activation L2: 
 [[0.9       0.7109495]] 

activation L3: 
 [[0.61405267]] 

error L3: 
 [[-0.09146642]] 

error L2: 
 [-0.01829328 -0.00751855] 

dJ_dw1_3:  -0.018293284971249293
dJ_dw2_3:  -0.0075185513677826785
